# Data generating of the 360-degree videos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import glob
from datetime import datetime as datetime
import time

In [2]:
# Configure panda to show all columns
pd.set_option('display.max_columns', None)

### Merging all the traces found

In [3]:
path = '../traces' # use your path
all_files = glob.glob(path + "/*.json")

li = []

for filename in all_files:
    df = pd.read_json(filename)
    li.append(df)

frame = pd.concat(li, ignore_index=True)


In [4]:
# Convert upload date (yyymmdd) to "days since upload"

current_time = datetime.now().timestamp()

for i in frame.index:
    try:
        upload_time = datetime.strptime(str(frame['upload_date'][i]), '%Y%m%d').timestamp()
        ms_since_upload = current_time - upload_time
        days_since_upload = ms_since_upload / 86400
        
        frame['upload_date'][i] =  days_since_upload
    except:
        pass

<ipython-input-4-262d4e5fef3c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['upload_date'][i] =  days_since_upload


In [5]:
gen_frame = frame.copy()
#gen_frame = frame.explode('formats')
#gen_frame = gen_frame.drop(['requested_formats', 'tags', 'thumbnails', 'formats', 'http_headers', "subtitles", 'automatic_captions', 'chapters', 'age_limit', 'annotations', 'average_rating', 'is_live', 'series', 'season_number', 'episode_number', 'release_date', 'release_year', 'playlist', 'playlist_index', 'requested_subtitles', 'stretched_ratio', 'preference', 'player_url'], axis=1)

gen_frame = gen_frame.explode('categories')
gen_frame = gen_frame[['upload_date','categories','duration', 'view_count','like_count','dislike_count', 'formats']]
music_frame = gen_frame[gen_frame['categories'] == "Music"]
music_frame = music_frame.explode('formats')
music_frame

,upload_date,categories,duration,view_count,like_count,dislike_count,formats
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '249', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '250', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '140', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '251', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '278', 'url': 'https://r5---sn-5..."
...,...,...,...,...,...,...,...
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '264', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '271', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '266', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '313', 'url': 'https://r1---sn-3..."


In [6]:
music_frame

,upload_date,categories,duration,view_count,like_count,dislike_count,formats
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '249', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '250', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '140', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '251', 'url': 'https://r5---sn-5..."
7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '278', 'url': 'https://r5---sn-5..."
...,...,...,...,...,...,...,...
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '264', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '271', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '266', 'url': 'https://r1---sn-3..."
2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '313', 'url': 'https://r1---sn-3..."


## Convert format objects to data columns

Format objects are JSON objects containing trace information. The properties (keys) of these objects must be unpacked into columns of the dataframe. This makes a row go from containing a column for one format object, to containing many columns; one for each property.

In [7]:
df = pd.DataFrame(music_frame['formats'].values.tolist())
music_frame = pd.concat([music_frame.reset_index(), df], axis =1)

music_frame

,index,upload_date,categories,duration,view_count,like_count,dislike_count,formats,format_id,url,player_url,ext,format_note,acodec,abr,asr,filesize,fps,height,tbr,width,vcodec,downloader_options,format,protocol,http_headers,container,manifest_url,language,fragment_base_url,fragments
0,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '249', 'url': 'https://r5---sn-5...",249,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,50.0,48000.0,1561009.0,NaN,NaN,66.067,NaN,none,{'http_chunk_size': 10485760},249 - audio only (tiny),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
1,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '250', 'url': 'https://r5---sn-5...",250,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,70.0,48000.0,2042379.0,NaN,NaN,83.013,NaN,none,{'http_chunk_size': 10485760},250 - audio only (tiny),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
2,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '140', 'url': 'https://r5---sn-5...",140,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,m4a,tiny,mp4a.40.2,128.0,44100.0,3637290.0,NaN,NaN,128.072,NaN,none,{'http_chunk_size': 10485760},140 - audio only (tiny),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,m4a_dash,NaN,NaN,NaN,NaN
3,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '251', 'url': 'https://r5---sn-5...",251,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,160.0,48000.0,3997269.0,NaN,NaN,154.814,NaN,none,{'http_chunk_size': 10485760},251 - audio only (tiny),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
4,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '278', 'url': 'https://r5---sn-5...",278,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,144s,none,NaN,NaN,2723218.0,30.0,144.0,151.809,256.0,vp9,{'http_chunk_size': 10485760},278 - 256x144 (144s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,webm,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '264', 'url': 'https://r1---sn-3...",264,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,1440s,none,NaN,NaN,266339756.0,30.0,1440.0,6336.801,2560.0,avc1.640032,{'http_chunk_size': 10485760},264 - 2560x1440 (1440s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3306,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '271', 'url': 'https://r1---sn-3...",271,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,1440s,none,NaN,NaN,204441926.0,30.0,1440.0,6511.530,2560.0,vp9,{'http_chunk_size': 10485760},271 - 2560x1440 (1440s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3307,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '266', 'url': 'https://r1---sn-3...",266,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,2160s,none,NaN,NaN,505069553.0,30.0,2160.0,11742.404,3840.0,avc1.640033,{'http_chunk_size': 10485760},266 - 3840x2160 (2160s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3308,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '313', 'url': 'https://r1---sn-3...",313,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,2160s,none,NaN,NaN,627878394.0,30.0,2160.0,17302.030,3840.0,vp9,{'http_chunk_size': 10485760},313 - 3840x2160 (2160s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN


### Filter out audio formats

We are only interested in video streaming. Audio-only formats are discarded to reduce runtime complexity and to obtain accurate results.

In [8]:
music_frame.reset_index(inplace=True)

music_frame = music_frame[music_frame["vcodec"] != "none"]

,index,upload_date,categories,duration,view_count,like_count,dislike_count,formats,format_id,url,player_url,ext,format_note,acodec,abr,asr,filesize,fps,height,tbr,width,vcodec,downloader_options,format,protocol,http_headers,container,manifest_url,language,fragment_base_url,fragments
4,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '278', 'url': 'https://r5---sn-5...",278,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,144s,none,NaN,NaN,2723218.0,30.0,144.0,151.809,256.0,vp9,{'http_chunk_size': 10485760},278 - 256x144 (144s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,webm,NaN,NaN,NaN,NaN
5,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '160', 'url': 'https://r5---sn-5...",160,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,144s,none,NaN,NaN,3172459.0,30.0,144.0,174.598,256.0,avc1.4d400c,{'http_chunk_size': 10485760},160 - 256x144 (144s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
6,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '242', 'url': 'https://r5---sn-5...",242,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,240s,none,NaN,NaN,5558170.0,30.0,240.0,259.172,426.0,vp9,{'http_chunk_size': 10485760},242 - 426x240 (240s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
7,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '133', 'url': 'https://r5---sn-5...",133,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,240s,none,NaN,NaN,6970471.0,30.0,240.0,356.027,426.0,avc1.4d4015,{'http_chunk_size': 10485760},133 - 426x240 (240s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
8,7,1736,Music,229,1242294,32096.0,510.0,"{'format_id': '243', 'url': 'https://r5---sn-5...",243,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,360s,none,NaN,NaN,10420173.0,30.0,360.0,507.340,640.0,vp9,{'http_chunk_size': 10485760},243 - 640x360 (360s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '264', 'url': 'https://r1---sn-3...",264,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,1440s,none,NaN,NaN,266339756.0,30.0,1440.0,6336.801,2560.0,avc1.640032,{'http_chunk_size': 10485760},264 - 2560x1440 (1440s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3306,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '271', 'url': 'https://r1---sn-3...",271,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,1440s,none,NaN,NaN,204441926.0,30.0,1440.0,6511.530,2560.0,vp9,{'http_chunk_size': 10485760},271 - 2560x1440 (1440s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3307,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '266', 'url': 'https://r1---sn-3...",266,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,2160s,none,NaN,NaN,505069553.0,30.0,2160.0,11742.404,3840.0,avc1.640033,{'http_chunk_size': 10485760},266 - 3840x2160 (2160s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN
3308,2927,2050,Music,404,6345840,59337.0,2203.0,"{'format_id': '313', 'url': 'https://r1---sn-3...",313,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,2160s,none,NaN,NaN,627878394.0,30.0,2160.0,17302.030,3840.0,vp9,{'http_chunk_size': 10485760},313 - 3840x2160 (2160s),https,{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ...,NaN,NaN,NaN,NaN,NaN


## Drop non-primitive (or string) data columns

We do this because the TGAN network can not evaluate/generate objects embedded in table cells.

In [9]:
tmp_frame = music_frame.convert_dtypes()
li = []
for col in tmp_frame:
    print(tmp_frame[col].dtype)
    if tmp_frame[col].dtype == 'object':
        li.append(col)
for item in li:
    music_frame = music_frame.drop(item, axis=1)

Int64
Int64
string
Int64
Int64
Int64
Int64
object
string
string
string
string
string
string
Int64
Int64
Int64
Int64
Int64
float64
Int64
string
object
string
string
object
string
string
Int64
string
object


In [10]:
music_frame

,index,upload_date,categories,duration,view_count,like_count,dislike_count,format_id,url,player_url,ext,format_note,acodec,abr,asr,filesize,fps,height,tbr,width,vcodec,format,protocol,container,manifest_url,language,fragment_base_url
0,7,1736,Music,229,1242294,32096.0,510.0,249,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,50.0,48000.0,1561009.0,NaN,NaN,66.067,NaN,none,249 - audio only (tiny),https,NaN,NaN,NaN,NaN
1,7,1736,Music,229,1242294,32096.0,510.0,250,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,70.0,48000.0,2042379.0,NaN,NaN,83.013,NaN,none,250 - audio only (tiny),https,NaN,NaN,NaN,NaN
2,7,1736,Music,229,1242294,32096.0,510.0,140,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,m4a,tiny,mp4a.40.2,128.0,44100.0,3637290.0,NaN,NaN,128.072,NaN,none,140 - audio only (tiny),https,m4a_dash,NaN,NaN,NaN
3,7,1736,Music,229,1242294,32096.0,510.0,251,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,tiny,opus,160.0,48000.0,3997269.0,NaN,NaN,154.814,NaN,none,251 - audio only (tiny),https,NaN,NaN,NaN,NaN
4,7,1736,Music,229,1242294,32096.0,510.0,278,https://r5---sn-5hnekn7k.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,144s,none,NaN,NaN,2723218.0,30.0,144.0,151.809,256.0,vp9,278 - 256x144 (144s),https,webm,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,2927,2050,Music,404,6345840,59337.0,2203.0,264,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,1440s,none,NaN,NaN,266339756.0,30.0,1440.0,6336.801,2560.0,avc1.640032,264 - 2560x1440 (1440s),https,NaN,NaN,NaN,NaN
3306,2927,2050,Music,404,6345840,59337.0,2203.0,271,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,1440s,none,NaN,NaN,204441926.0,30.0,1440.0,6511.530,2560.0,vp9,271 - 2560x1440 (1440s),https,NaN,NaN,NaN,NaN
3307,2927,2050,Music,404,6345840,59337.0,2203.0,266,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,mp4,2160s,none,NaN,NaN,505069553.0,30.0,2160.0,11742.404,3840.0,avc1.640033,266 - 3840x2160 (2160s),https,NaN,NaN,NaN,NaN
3308,2927,2050,Music,404,6345840,59337.0,2203.0,313,https://r1---sn-32o-guhz.googlevideo.com/video...,/s/player/408be03a/player_ias.vflset/en_US/bas...,webm,2160s,none,NaN,NaN,627878394.0,30.0,2160.0,17302.030,3840.0,vp9,313 - 3840x2160 (2160s),https,NaN,NaN,NaN,NaN


### Drop irrelevant columns

To reduce runtime complexity and increase the accuracy of the results, we drop columns 

In [11]:
from sdv.tabular import CTGAN
model = CTGAN()
model.fit(music_frame)
new_data = model.sample(50)
new_data.describe()

ModuleNotFoundError: No module named 'sdv'

In [ ]:
new_data = model.sample(2000)
new_data.head()

### Inspect data

In [ ]:
frame.head(2)

In [ ]:
len(frame.id.unique())

In [ ]:
frame.describe()

In [ ]:
print(df.columns.tolist())

In [ ]:
len(frame['categories'].apply(sorted).transform(tuple).unique())

In [ ]:
categories = pd.DataFrame(frame['categories'].apply(sorted).transform(tuple).unique())

In [ ]:
categories

In [ ]:
grouped = frame.groupby("categories")

In [ ]:
newFrame = frame['categories'].apply(sorted).transform(tuple)

In [ ]:
frame["tupleCat"] = newFrame

In [ ]:
frame["tupleCat"]

In [ ]:
frame.groupby("tupleCat").agg("count")["id"]

Note: make a bar chart of category video count

## Plotting relations between categories and other characteristics

### Upload date

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['upload_date'], ax=ax, grid=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.get_yaxis().set_ticks([365, 730, 1095, 1460, 1825])

### Popularity

#### Plot popularity

Note: like/dislike ratio is not being used for calculating the popularity of a video (see paragraph under graphs).

In [ ]:
# For each entry, calculate its popularity
frame['popularity'] = 0.0 # initial float value
frame['like_dislike_ratio'] = 0.0

for i in frame.index:
    view_count = frame['view_count'][i]
    days_since_upload = frame['upload_date'][i]
    like_count = frame['like_count'][i]
    dislike_count = frame['dislike_count'][i]

    like_dislike_ratio = like_count / dislike_count

    popularity = (view_count / float(days_since_upload))# * like_dislike_ratio

    frame['like_dislike_ratio'][i] = like_dislike_ratio
    frame['popularity'][i] =  popularity

In [ ]:
# Plot popularity

fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['popularity'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### Plot like/dislike ratio

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
frame.explode('categories').boxplot(by='categories', column=['like_dislike_ratio'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.get_yaxis().set_ticks([1, 10])

In the boxplot above, we find that the ratio between likes and dislikes on a video is hardly ever below 1. Therefore, we may conclude that viewers are more inclined to indicate which videos they like than to indicate which videos they dislike. And indeed, content creators usually encourage their audience to like their videos. Therefore, the ratio between likes and dislikes seems to be positively dominated by a relatively high number of likes.

This means that the like/dislike ratio may overrepresent the positive perception and we must be careful with using this metric.

### Available representations

#### Number of representations

Not solved yet.

In [ ]:
df1 = (pd.concat({i: pd.DataFrame(x) for i, x in frame.pop('formats').items()})
         .reset_index(level=1, drop=True)
         .join(frame, rsuffix='_shared')
         .reset_index(drop=True))

df1 = df1[df1.vcodec != "none"]

df1['available_representations'] = df1['formats'].str.len()

fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['available_representations'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

In [ ]:
# df1 = (pd.concat({i: pd.DataFrame(x) for i, x in frame.pop('formats').items()})
#          .reset_index(level=1, drop=True)
#          .join(frame, rsuffix='_shared')
#          .reset_index(drop=True))

# df1 = df1[df1.vcodec != "none"]

#### Average bitrate

In [ ]:
# Create column for average bitrate (kbps)
df1['average_bitrate'] = 0.0

for i in df1.index:
    try:
        file_size_bytes = df1['filesize'][i]
        file_size_bits = file_size_bytes * 8
        
        duration = df1['duration'][i]
        
        average_bitrate = (file_size_bits / float(duration)) / 1000 # average bitrate in kbps
        
        df1['average_bitrate'][i] =  average_bitrate
    except:
        pass

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['average_bitrate'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### File types

In [ ]:
df1.groupby('categories')

In [ ]:
df1.groupby('ext').agg('count')["id"]

#### Framerates

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['fps'], ax=ax, grid=False, showfliers=True)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')

#### Durations

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
df1.explode('categories').boxplot(by='categories', column=['duration'], ax=ax, grid=False, showfliers=False)

ax.yaxis.grid(which='major', linestyle='-', linewidth='0.5', color='red')